In [1]:
# Simple example for generating predictions with MHCflurry.
import mhcflurry

In [2]:
# Load a predictor
predictor = mhcflurry.Class1PresentationPredictor.load()
predictor

Using TensorFlow backend.
W0605 12:25:31.131366 4512298432 deprecation.py:506] From /Users/tim/miniconda3/envs/py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<Class1PresentationPredictor at 0x148588128 [mhcflurry 1.7.0] generated on Fri May  1 11:08:06 2020>

In [3]:
# Predict for individiual peptides.
# Try help(predictor.predict) for other options.
results1 = predictor.predict(["NLVPMVATV", "KLLEIPDPDKNWATL", "RANDMPEPTIDE"], ["A*02:01"])
results1

  0%|          | 0/1 [00:00<?, ?it/s]

Predicting processing.


W0605 12:25:34.682682 4512298432 module_wrapper.py:139] From /Users/tim/miniconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

  0%|          | 0/1 [00:00<?, ?it/s]

Predicting affinities.


100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


peptide  peptide_num sample_name      affinity best_allele  \
0        NLVPMVATV            0     sample1     23.634000     A*02:01   
1  KLLEIPDPDKNWATL            1     sample1   1539.483711     A*02:01   
2     RANDMPEPTIDE            2     sample1  18589.200876     A*02:01   

   processing_score  presentation_score  presentation_percentile  
0          0.456810            0.990288                 0.010220  
1          0.605072            0.538179                 0.754209  
2          0.006360            0.007214                59.896462

In [4]:
# Predict across protein sequences
# Try help(predictor.predict_sequences) for other options.
results2 = predictor.predict_sequences(
    sequences={
        'protein1': "MDSKGSSQKGSRLLLLLVVSNLL",
        'protein2': "SSLPTPEDKEQAQQTHH",
    },
    alleles={
        "sample1": ["A0201", "A0301", "B0702"],
        "sample2": ["A0101", "C0202"],
    },
    result="filtered",
    comparison_quantity="affinity",
    filter_value=500)
results2

  0%|          | 0/1 [00:00<?, ?it/s]

Predicting processing.


 40%|████      | 2/5 [00:00<00:00, 18.32it/s]

Predicting affinities.


100%|██████████| 5/5 [00:00<00:00, 19.89it/s]


sequence_name  pos    peptide         n_flank c_flank sample_name  \
0      protein1   14  LLLVVSNLL  MDSKGSSQKGSRLL             sample1   
1      protein1   13  LLLLVVSNL   MDSKGSSQKGSRL       L     sample1   

     affinity best_allele  affinity_percentile  processing_score  \
0  119.641959       A0201             0.428125          0.103990   
1  165.667454       A0201             0.513375          0.028328   

   presentation_score  presentation_percentile  
0            0.790948                 0.328758  
1            0.656283                 0.540264

In [ ]:

import csv
import pandas as pd

fileslist = ["1-MHCFLURRY.csv","2-MHCFLURRY.csv","3-MHCFLURRY.csv","4-MHCFLURRY.csv"]

for files in fileslist:
    f = open(files)
    csv_f = csv.reader(f)
    list_pept = []
    list_allel = [] 
    result = []
    error = []
    resultFilename = 'Result' + "-" + files
    errorFilename = 'Error' + "-" + files
    file = open(resultFilename, "a")
    efile = open(errorFilename, "a")
    for row in csv_f:
        #remove HLA-A, add* after first letter
        line = str.split(row[0])
        print(line)
        list_allel.append(line[1])
        list_pept.append(line[0])
        try:
            prediction=(predictor.predict([line[0]], [line[1]]))
            result.append(prediction)
            data = prediction.to_csv()
            #print(prediction to file)
            file.write(data)
        except Exception as e:
            error_data = {'HLA':[line[1]],'Peptide':[line[0]], 'ErrorMsg':[str(e)]}
            # Create DataFrame
            df = pd.DataFrame(error_data)
            error.append(df)
            #print(error to file)
            efile.write(df.to_csv())
            
    file.close()
    efile.close()